In [1]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, StringIndexer
from pyspark.sql.functions import length, size, lit
from pyspark.sql import SQLContext
import string
sqlContext = SQLContext(sc)


In [2]:
df = spark.table("all_csv")

df.columns

Out[ 2 ]: ['_c0', 'username', 'Gender', 'Education', 'Ethnicity', 'about_me_text_clean']

In [3]:
df = df.select('username', 'about_me_text_clean', 'Gender').na.drop()
df_length = df.withColumn('length', length(df['about_me_text_clean']))

male_female_to_num = StringIndexer(inputCol='Gender', outputCol='label')

df = male_female_to_num.fit(df_length).transform(df_length)
df.show()

+--------------+--------------------+------+------+-----+
 username| about_me_text_clean|Gender|length|label|
+--------------+--------------------+------+------+-----+
 ROXYCHIC549| I am not an upgr...| F| 1615| 0.0|
 tequilagirl69|Laid back lovable...| F| 175| 0.0|
atxredneckgirl|Tx country music ...| F| 259| 0.0|
 mel12th|I am here to look...| F| 338| 0.0|
 Andi010274|Cant get into my ...| F| 47| 0.0|
 nodutch|Music business go...| F| 67| 0.0|
 lauranmr|Of all the dating...| F| 708| 0.0|
 smbd75|Likes fun adventu...| F| 66| 0.0|
 candipie1989|Single black fema...| F| 376| 0.0|
 tif1985|Im on here just l...| F| 300| 0.0|
 De1964|Please local only...| F| 1088| 0.0|
 Trini1971|I am a Mom to a 1...| F| 564| 0.0|
 FiaFia1213|I like to think I...| F| 341| 0.0|
 SweetJojos| Up for anything| F| 15| 0.0|
 Tawdra83|I’m a artist I co...| F| 423| 0.0|
 LadyTexas254|not looking for a...| F| 233| 0.0|
 Vigilia79|Hi there Im Chris...| F| 607| 0.0|
 ayoung292|Music is life and...| F| 102| 0.0|
 Foreverfungrl| Ask me Are| F| 10| 0.0|
 southstar5|Single have two k...| F| 39| 0.0|
+--------------+--------------------+------+------+-----+
only showing top 20 rows

In [4]:
# Tokenize data
tokened_df = Tokenizer(inputCol="about_me_text_clean", outputCol="token_text")
tt_df = tokened_df.transform(df)
tt_df = tt_df.where(size(tt_df["token_text"]) > 10)
tt_df = tt_df.where(tt_df['label'] < 2.0)

tt_df.show()


+-----------------+--------------------+------+------+-----+--------------------+
 username| about_me_text_clean|Gender|length|label| token_text|
+-----------------+--------------------+------+------+-----+--------------------+
 ROXYCHIC549| I am not an upgr...| F| 1615| 0.0|[, i, am, not, an...|
 tequilagirl69|Laid back lovable...| F| 175| 0.0|[laid, back, lova...|
 atxredneckgirl|Tx country music ...| F| 259| 0.0|[tx, country, mus...|
 mel12th|I am here to look...| F| 338| 0.0|[i, am, here, to,...|
 Andi010274|Cant get into my ...| F| 47| 0.0|[cant, get, into,...|
 lauranmr|Of all the dating...| F| 708| 0.0|[of, all, the, da...|
 candipie1989|Single black fema...| F| 376| 0.0|[single, black, f...|
 tif1985|Im on here just l...| F| 300| 0.0|[im, on, here, ju...|
 De1964|Please local only...| F| 1088| 0.0|[please, local, o...|
 Trini1971|I am a Mom to a 1...| F| 564| 0.0|[i, am, a, mom, t...|
 FiaFia1213|I like to think I...| F| 341| 0.0|[i, like, to, thi...|
 Tawdra83|I’m a artist I co...| F| 423| 0.0|[i’m, a, artist, ...|
 LadyTexas254|not looking for a...| F| 233| 0.0|[not, looking, fo...|
 Vigilia79|Hi there Im Chris...| F| 607| 0.0|[hi, there, im, c...|
 ayoung292|Music is life and...| F| 102| 0.0|[music, is, life,...|
 71NOexpectations|Just testing the ...| F| 63| 0.0|[just, testing, t...|
 LadytexanInSA|Hi I am a single ...| F| 1200| 0.0|[hi, i, am, a, si...|
Wishing_on_stars1|Looking for the w...| F| 529| 0.0|[looking, for, th...|
 KarmaChris|I know my worth a...| F| 120| 0.0|[i, know, my, wor...|
 cat2018bf|Im a mother of 4 ...| F| 147| 0.0|[im, a, mother, o...|
+-----------------+--------------------+------+------+-----+--------------------+
only showing top 20 rows

In [5]:
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
removed_frame = stop_remove.transform(tt_df)
removed_frame.show()

+-----------------+--------------------+------+------+-----+--------------------+--------------------+
 username| about_me_text_clean|Gender|length|label| token_text| stop_tokens|
+-----------------+--------------------+------+------+-----+--------------------+--------------------+
 ROXYCHIC549| I am not an upgr...| F| 1615| 0.0|[, i, am, not, an...|[, upgraded, memb...|
 tequilagirl69|Laid back lovable...| F| 175| 0.0|[laid, back, lova...|[laid, back, lova...|
 atxredneckgirl|Tx country music ...| F| 259| 0.0|[tx, country, mus...|[tx, country, mus...|
 mel12th|I am here to look...| F| 338| 0.0|[i, am, here, to,...|[look, people, co...|
 Andi010274|Cant get into my ...| F| 47| 0.0|[cant, get, into,...|[cant, get, accou...|
 lauranmr|Of all the dating...| F| 708| 0.0|[of, all, the, da...|[dating, profiles...|
 candipie1989|Single black fema...| F| 376| 0.0|[single, black, f...|[single, black, f...|
 tif1985|Im on here just l...| F| 300| 0.0|[im, on, here, ju...|[im, looking, fri...|
 De1964|Please local only...| F| 1088| 0.0|[please, local, o...|[please, local, c...|
 Trini1971|I am a Mom to a 1...| F| 564| 0.0|[i, am, a, mom, t...|[mom, 15yearold, ...|
 FiaFia1213|I like to think I...| F| 341| 0.0|[i, like, to, thi...|[like, think, mos...|
 Tawdra83|I’m a artist I co...| F| 423| 0.0|[i’m, a, artist, ...|[i’m, artist, coo...|
 LadyTexas254|not looking for a...| F| 233| 0.0|[not, looking, fo...|[looking, anythin...|
 Vigilia79|Hi there Im Chris...| F| 607| 0.0|[hi, there, im, c...|[hi, im, christin...|
 ayoung292|Music is life and...| F| 102| 0.0|[music, is, life,...|[music, life, con...|
 71NOexpectations|Just testing the ...| F| 63| 0.0|[just, testing, t...|[testing, waters,...|
 LadytexanInSA|Hi I am a single ...| F| 1200| 0.0|[hi, i, am, a, si...|[hi, single, hisp...|
Wishing_on_stars1|Looking for the w...| F| 529| 0.0|[looking, for, th...|[looking, whole, ...|
 KarmaChris|I know my worth a...| F| 120| 0.0|[i, know, my, wor...|[know, worth, don...|
 cat2018bf|Im a mother of 4 ...| F| 147| 0.0|[im, a, mother, o...|[im, mother, 4, a...|
+-----------------+--------------------+------+------+-----+--------------------+--------------------+
only showing top 20 rows

In [6]:
# Run the hashing term frequency
hashing = HashingTF(inputCol='token_text', outputCol="hashedValues")

# Transform into a DF
hashed_df = hashing.transform(removed_frame)
hashed_df.show()

+-----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+
 username| about_me_text_clean|Gender|length|label| token_text| stop_tokens| hashedValues|
+-----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+
 ROXYCHIC549| I am not an upgr...| F| 1615| 0.0|[, i, am, not, an...|[, upgraded, memb...|(262144,[4200,538...|
 tequilagirl69|Laid back lovable...| F| 175| 0.0|[laid, back, lova...|[laid, back, lova...|(262144,[4333,844...|
 atxredneckgirl|Tx country music ...| F| 259| 0.0|[tx, country, mus...|[tx, country, mus...|(262144,[7491,963...|
 mel12th|I am here to look...| F| 338| 0.0|[i, am, here, to,...|[look, people, co...|(262144,[14,7401,...|
 Andi010274|Cant get into my ...| F| 47| 0.0|[cant, get, into,...|[cant, get, accou...|(262144,[6693,244...|
 lauranmr|Of all the dating...| F| 708| 0.0|[of, all, the, da...|[dating, profiles...|(262144,[1998,761...|
 candipie1989|Single black fema...| F| 376| 0.0|[single, black, f...|[single, black, f...|(262144,[8449,134...|
 tif1985|Im on here just l...| F| 300| 0.0|[im, on, here, ju...|[im, looking, fri...|(262144,[4200,576...|
 De1964|Please local only...| F| 1088| 0.0|[please, local, o...|[please, local, c...|(262144,[3326,408...|
 Trini1971|I am a Mom to a 1...| F| 564| 0.0|[i, am, a, mom, t...|[mom, 15yearold, ...|(262144,[9639,119...|
 FiaFia1213|I like to think I...| F| 341| 0.0|[i, like, to, thi...|[like, think, mos...|(262144,[329,9639...|
 Tawdra83|I’m a artist I co...| F| 423| 0.0|[i’m, a, artist, ...|[i’m, artist, coo...|(262144,[2437,963...|
 LadyTexas254|not looking for a...| F| 233| 0.0|[not, looking, fo...|[looking, anythin...|(262144,[9988,133...|
 Vigilia79|Hi there Im Chris...| F| 607| 0.0|[hi, there, im, c...|[hi, im, christin...|(262144,[4054,420...|
 ayoung292|Music is life and...| F| 102| 0.0|[music, is, life,...|[music, life, con...|(262144,[15889,24...|
 71NOexpectations|Just testing the ...| F| 63| 0.0|[just, testing, t...|[testing, waters,...|(262144,[5360,158...|
 LadytexanInSA|Hi I am a single ...| F| 1200| 0.0|[hi, i, am, a, si...|[hi, single, hisp...|(262144,[2437,517...|
Wishing_on_stars1|Looking for the w...| F| 529| 0.0|[looking, for, th...|[looking, whole, ...|(262144,[3067,104...|
 KarmaChris|I know my worth a...| F| 120| 0.0|[i, know, my, wor...|[know, worth, don...|(262144,[9639,244...|
 cat2018bf|Im a mother of 4 ...| F| 147| 0.0|[im, a, mother, o...|[im, mother, 4, a...|(262144,[9639,158...|
+-----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+
only showing top 20 rows

In [7]:
# Fit the IDF on the data set 
idf = IDF(minDocFreq = 10, inputCol="hashedValues", outputCol="idf_features")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [8]:
# Display rescaled dataframe
#rescaledData.show()
rescaledData.select("token_text", "idf_features").show()

+--------------------+--------------------+
 token_text| idf_features|
+--------------------+--------------------+
[, i, am, not, an...|(262144,[4200,538...|
[laid, back, lova...|(262144,[4333,844...|
[tx, country, mus...|(262144,[7491,963...|
[i, am, here, to,...|(262144,[14,7401,...|
[cant, get, into,...|(262144,[6693,244...|
[of, all, the, da...|(262144,[1998,761...|
[single, black, f...|(262144,[8449,134...|
[im, on, here, ju...|(262144,[4200,576...|
[please, local, o...|(262144,[3326,408...|
[i, am, a, mom, t...|(262144,[9639,119...|
[i, like, to, thi...|(262144,[329,9639...|
[i’m, a, artist, ...|(262144,[2437,963...|
[not, looking, fo...|(262144,[9988,133...|
[hi, there, im, c...|(262144,[4054,420...|
[music, is, life,...|(262144,[15889,24...|
[just, testing, t...|(262144,[5360,158...|
[hi, i, am, a, si...|(262144,[2437,517...|
[looking, for, th...|(262144,[3067,104...|
[i, know, my, wor...|(262144,[9639,244...|
[im, a, mother, o...|(262144,[9639,158...|
+--------------------+--------------------+
only showing top 20 rows

In [9]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_features', 'length'], outputCol='features')
cleaned_data = clean_up.transform(rescaledData)

In [10]:
#display cleaned data
cleaned_data.select(['label', 'idf_features']).show()

+-----+--------------------+
label| idf_features|
+-----+--------------------+
 0.0|(262144,[4200,538...|
 0.0|(262144,[4333,844...|
 0.0|(262144,[7491,963...|
 0.0|(262144,[14,7401,...|
 0.0|(262144,[6693,244...|
 0.0|(262144,[1998,761...|
 0.0|(262144,[8449,134...|
 0.0|(262144,[4200,576...|
 0.0|(262144,[3326,408...|
 0.0|(262144,[9639,119...|
 0.0|(262144,[329,9639...|
 0.0|(262144,[2437,963...|
 0.0|(262144,[9988,133...|
 0.0|(262144,[4054,420...|
 0.0|(262144,[15889,24...|
 0.0|(262144,[5360,158...|
 0.0|(262144,[2437,517...|
 0.0|(262144,[3067,104...|
 0.0|(262144,[9639,244...|
 0.0|(262144,[9639,158...|
+-----+--------------------+
only showing top 20 rows

In [11]:
from pyspark.ml.classification import NaiveBayes, NaiveBayesModel
# Break data down into a training set and a testing set
#training, testing = cleaned_data.randomSplit([0.7, 0.3], seed = 42)
fractions = cleaned_data.select("label").distinct().withColumn("fraction", lit(0.7)).rdd.collectAsMap()

training,testing = cleaned_data.stat.sampleBy("label", fractions, 12).randomSplit([0.7, 0.3])
#testing = cleaned_data.subtract(training)

# Create a Naive Bayes model and fit training data
nb = NaiveBayes(smoothing = 1)
predictor = nb.fit(training)


In [12]:
test_results = predictor.transform(testing)
test_results.show(3)

+-------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 username| about_me_text_clean|Gender|length|label| token_text| stop_tokens| hashedValues| idf_features| features| rawPrediction| probability|prediction|
+-------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 1KindOfLove|Hi my name is She...| F| 854| 0.0|[hi, my, name, is...|[hi, name, sheila...|(262144,[9639,144...|(262144,[9639,144...|(262145,[9639,144...|[-3366.2429723296...|[1.0,2.3112209182...| 0.0|
1simplysweet1|I’m an introvert ...| F| 565| 0.0|[i’m, an, introve...|[i’m, introvert, ...|(262144,[8443,963...|(262144,[8443,963...|(262145,[8443,963...|[-2056.0131998765...|[1.0,4.7938442902...| 0.0|
 210Frances|Hello My name is ...| F| 229| 0.0|[hello, my, name,...|[hello, name, fra...|(262144,[8443,158...|(262144,[8443,158...|(262145,[8443,158...|[-840.63769174300...|[0.99999958340870...| 0.0|
+-------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 3 rows

In [13]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting gender was: %f" % acc)

Accuracy of model at predicting gender was: 0.476916